In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, codeine_idx, measurement_type_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_codeine = all_complete.data[codeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (15,19,23,24,25,28,29,30,31,44,72,88,90,114,115,117,119,121,123,124,126,127,128,129,132,133,134,136,140,144,145,147,148,149,151,152,172,174,175,176,177,179,180,182,184,186,188,194,195,196,203,204,208,211,212,220,225,229,230,235,239) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
codeine_clearance = measurement_type_data(complete_codeine,"clearance")

codeine_clearance[("weight","value")] = pd.to_numeric(codeine_clearance[("weight","value")])
codeine_clearance[("weight","mean")] = pd.to_numeric(codeine_clearance[("weight","mean")])
codeine_clearance["value_intervention"] = pd.to_numeric(codeine_clearance["value_intervention"])
codeine_clearance["median"] = pd.to_numeric(codeine_clearance["median"])
codeine_clearance["mean"] = pd.to_numeric(codeine_clearance["mean"])
codeine_clearance["value"] = pd.to_numeric(codeine_clearance["value"])
codeine_clearance["se"] = pd.to_numeric(codeine_clearance["se"])
codeine_clearance["sd"] = pd.to_numeric(codeine_clearance["sd"])
codeine_clearance["cv"] = pd.to_numeric(codeine_clearance["cv"])

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [7]:
codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/hr",
                             unit_out="l/hr/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs_i = convert_unit(codeine_clearance,
                             unit_in="l/hr/kg",
                             unit_out="l/hr", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs_i], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)


codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

In [8]:
codeine_clr = PkdbModel("codeine_clearance",destination="3-inferred")
codeine_clr.data = codeine_clearance
codeine_clr.save()
codeine_clearance.groupby(["unit","unit_intervention"]).count()

,,study,output_type,pk,access,measurement_type,tissue,allowed_users,substance,interventions,raw_pk,...,"(cyp2d6 variant, choice)","(cyp2d6 variant, count)","(lean body mass, count)","(lean body mass, mean)","(lean body mass, sd)","(lean body mass, unit)","(fat weight, count)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)"
unit,unit_intervention,,,,,,,,,,,,,,,,,,,,,
liter / hour,gram,23,23,23,23,23,23,23,23,23,23,...,0,0,0,0,0,0,0,0,0,0


## Timecourses

In [9]:
codeine_timecourse = measurement_type_data(complete_codeine,"concentration")
codeine_timecourse = filter_out(codeine_timecourse,"output_type",["outputs"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/l"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/ml"])

codeine_timecourse.groupby(["unit","unit_intervention"]).count()

,,study,output_type,pk,access,measurement_type,tissue,allowed_users,substance,interventions,raw_pk,...,"(cyp2d6 variant, choice)","(cyp2d6 variant, count)","(lean body mass, count)","(lean body mass, mean)","(lean body mass, sd)","(lean body mass, unit)","(fat weight, count)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)"
unit,unit_intervention,,,,,,,,,,,,,,,,,,,,,
gram / liter,gram,6,6,6,6,6,6,6,6,6,6,...,0,0,0,0,0,0,0,0,0,0
mole / liter,gram,11,11,11,11,11,11,11,11,11,11,...,0,0,0,0,0,0,0,0,0,0


In [10]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.to_array()
codeine_timecourse = codeine_tc.data

In [11]:
codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])



codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

In [12]:
codeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

,,,study,output_type,pk,access,measurement_type,tissue,allowed_users,substance,interventions,raw_pk,...,"(cyp2d6 variant, choice)","(cyp2d6 variant, count)","(lean body mass, count)","(lean body mass, mean)","(lean body mass, sd)","(lean body mass, unit)","(fat weight, count)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)"
unit,unit_intervention,time_unit,,,,,,,,,,,,,,,,,,,,,
gram / liter,gram,hr,6,6,6,6,6,6,6,6,6,6,...,0,0,0,0,0,0,0,0,0,0
mole / liter,gram,hr,11,11,11,11,11,11,11,11,11,11,...,0,0,0,0,0,0,0,0,0,0


In [13]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.save()